In [1]:
from prometheus_client import start_http_server, Gauge
import random
import serial
import time
import json

In [2]:
# read configurator file and import all varibales
with open("exporterConfigrator.json", "r") as file:
        json_data = file.read()
data_dict = json.loads(json_data)
# sensorName and portNo will be automatically read from the json file.
data_dict["metric_tags"]

{'rasPi': 'EMON-R22',
 'microController': 'Master-RPICTV7v5.7',
 'sensorName': '',
 'portNo': ''}

In [3]:
# create a metric for current 
new_metric = data_dict["current_metric"]
SENSOR_CURRENT = Gauge (
    new_metric["metric_name"], 
    new_metric["metric_description"], 
    list(data_dict["metric_tags"].keys())
)

# create a metric for power
new_metric = data_dict["power_metric"]
SENSOR_POWER = Gauge (
    new_metric["metric_name"], 
    new_metric["metric_description"], 
    list(data_dict["metric_tags"].keys())
)

# create a metric for voltage 
new_metric = data_dict["voltage_metric"]
SENSOR_VOLTAGE = Gauge (
    new_metric["metric_name"], 
    new_metric["metric_description"], 
    list(data_dict["metric_tags"].keys())
)

type(SENSOR_CURRENT), type(SENSOR_POWER), type(SENSOR_VOLTAGE)

(prometheus_client.metrics.Gauge,
 prometheus_client.metrics.Gauge,
 prometheus_client.metrics.Gauge)

In [4]:
# find the names and description of all active ports
def port_mapper(indexes):
    # Convert JSON data to Python dictionary
    ports = data_dict["portMapper"]
    # Retrieve values for the given indexes
    names = []
    descriptions = []
    for index in indexes:
        port_data = ports.get(str(index))
        if port_data:
            names.append(port_data["name"])
            descriptions.append(port_data["description"])
        else:
            names.append(None)
            descriptions.append(None)
    return names, descriptions

# Decorate function with metric.
def process_request(active_ports_data):
    """A function that exposes the information of all active ports"""
    # First, construct the active_ports_data dictionary as a dictionary of dictionaries where each port name is a key,
    # and the corresponding value is another dictionary containing the port value and description:
    active_ports_data = {}
    for name, value, description in zip(active_ports_names, active_ports_values, port_description):
        active_ports_data[name] = {'value': value, 'description': description}

    #  iterate through the active_ports_data dictionary as
    for port_name, port_data in active_ports_data.items():
        print(port_name, port_data['value'], port_data['description'])  
        if port_name.endswith('A'):
            SENSOR_CURRENT.labels(
                rasPi= data_dict["metric_tags"]["rasPi"], 
                microController=data_dict["metric_tags"]["microController"], 
                sensorName=port_data['description'], 
                portNo=port_name
            ).set(port_data['value'])
        elif port_name.endswith('P'):
            SENSOR_POWER.labels(
                rasPi= data_dict["metric_tags"]["rasPi"], 
                microController=data_dict["metric_tags"]["microController"], 
                sensorName=port_data['description'], 
                portNo=port_name
            ).set(port_data['value'])
        elif port_name.endswith('e'):
            SENSOR_VOLTAGE.labels(
                rasPi= data_dict["metric_tags"]["rasPi"], 
                microController=data_dict["metric_tags"]["microController"], 
                sensorName=port_data['description'], 
                portNo=port_name
            ).set(port_data['value'])
    time.sleep(0.10)    # waiting time before next execuation

In [ ]:
if __name__ == '__main__':
    # Start up the server to expose the metrics at port number mention below.
    start_http_server(8000)
    # port name of sensor to read data from
    ser = serial.Serial('/dev/ttyAMA0', 38400)
    try:
        while 1:
            # Read one line from the serial buffer
            line = ser.readline().decode().strip()
            Z = line.split(' ')                          # Create an array of the data
            if len(Z)>15:                               # Print it nicely
                # --------- START: PRINT ON SCREEN ---------
                print ("----------")
                print ("Vrms:\t%s" % Z[15])
                print ("          \tCT1\tCT2\tCT3\tCT4\tCT5\tCT6\tCT7")
                print ("RealPower:\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % (Z[1], Z[2], Z[3], Z[4], Z[5], Z[6], Z[7]))
                print ("Irms     :\t%s\t%s\t%s\t%s\t%s\t%s\t%s" % (Z[8], Z[9], Z[10], Z[11], Z[12], Z[13], Z[14]))
                # --------- END: PRINT ON SCREEN ---------

                # Refer to port_mapper.json file for port and indexes numbers.
                active_ports_indexes = data_dict["activePort"]         # create a list of active port indexes
                active_ports_names, port_description = port_mapper(active_ports_indexes) # port name 
                active_ports_values  = [Z[i] for i in active_ports_indexes]
                active_ports_data = dict(zip(active_ports_names, active_ports_values))
                # active_ports_data is a dictionary where each key is a port name, and the corresponding value
                # is a dictionary containing the port value and description
                for name, description in zip(active_ports_names, port_description):
                    active_ports_data[name] = {'value': active_ports_data[name], 'description': description}

                process_request(active_ports_data)

    except KeyboardInterrupt:
        ser.close()

----------
Vrms:	257.49
          	CT1	CT2	CT3	CT4	CT5	CT6	CT7
RealPower:	-0.06	0.14	-0.04	-0.09	0.02	185.75	-216.59
Irms     :	0.014	0.013	0.013	0.013	0.014	0.845	0.942
CT6P 185.75 CSG-2
CT7P -216.59 CSG-1
CT6A 0.845 CSG-2
CT7A 0.942 CSG-1
Voltage 257.49 Electric-Socket
----------
Vrms:	257.49
          	CT1	CT2	CT3	CT4	CT5	CT6	CT7
RealPower:	-0.02	0.00	0.01	-0.00	-0.05	192.27	-214.99
Irms     :	0.012	0.013	0.012	0.012	0.013	0.879	0.936
CT6P 192.27 CSG-2
CT7P -214.99 CSG-1
CT6A 0.879 CSG-2
CT7A 0.936 CSG-1
Voltage 257.49 Electric-Socket
----------
Vrms:	257.43
          	CT1	CT2	CT3	CT4	CT5	CT6	CT7
RealPower:	0.01	0.07	0.04	-0.09	nan	184.55	-215.04
Irms     :	0.013	0.013	0.013	0.013	nan	0.852	0.936
CT6P 184.55 CSG-2
CT7P -215.04 CSG-1
CT6A 0.852 CSG-2
CT7A 0.936 CSG-1
Voltage 257.43 Electric-Socket
----------
Vrms:	258.12
          	CT1	CT2	CT3	CT4	CT5	CT6	CT7
RealPower:	-0.06	-0.04	-0.08	-0.01	-0.02	nan	-218.31
Irms     :	0.014	0.011	0.013	0.014	0.012	nan	0.945
CT6P nan CSG-2
CT7P -2